# Chapter 10
## Data Aggregation and Group Operation

## 10.1 GroupBy Mechanics

In [3]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'key1': list('aabba'),
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df

,data1,data2,key1,key2
0,0.912406,1.245481,a,one
1,-0.410569,-1.398004,a,two
2,-0.680363,-0.760848,b,one
3,1.203681,-0.477818,b,two
4,-0.959425,-0.902707,a,one


In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [5]:
grouped.mean()

key1
a   -0.152529
b    0.261659
Name: data1, dtype: float64

In [7]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.023510
      two    -0.410569
b     one    -0.680363
      two     1.203681
Name: data1, dtype: float64

In [8]:
means.unstack()

key2,one,two
key1,,
a,-0.023510,-0.410569
b,-0.680363,1.203681


In [9]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -0.410569
            2006   -0.680363
Ohio        2005    1.058043
            2006   -0.959425
Name: data1, dtype: float64

In [10]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.152529,-0.351743
b,0.261659,-0.619333


In [11]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.023510  0.171387
     two  -0.410569 -1.398004
b    one  -0.680363 -0.760848
     two   1.203681 -0.477818

In [12]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [15]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0  0.912406  1.245481    a  one
1 -0.410569 -1.398004    a  two
4 -0.959425 -0.902707    a  one
b
      data1     data2 key1 key2
2 -0.680363 -0.760848    b  one
3  1.203681 -0.477818    b  two


In [17]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
      data1     data2 key1 key2
0  0.912406  1.245481    a  one
4 -0.959425 -0.902707    a  one
('a', 'two')
      data1     data2 key1 key2
1 -0.410569 -1.398004    a  two
('b', 'one')
      data1     data2 key1 key2
2 -0.680363 -0.760848    b  one
('b', 'two')
      data1     data2 key1 key2
3  1.203681 -0.477818    b  two


In [19]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,data1,data2,key1,key2
2,-0.680363,-0.760848,b,one
3,1.203681,-0.477818,b,two


In [26]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [27]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group, in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.912406  1.245481
1 -0.410569 -1.398004
2 -0.680363 -0.760848
3  1.203681 -0.477818
4 -0.959425 -0.902707
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

In [29]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
# ... are equivalent to:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [34]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.171387
     two  -1.398004
b    one  -0.760848
     two  -0.477818

In [35]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [36]:
s_grouped.mean()

key1  key2
a     one     0.171387
      two    -1.398004
b     one    -0.760848
      two    -0.477818
Name: data2, dtype: float64

### Grouping with Dicts and Series

In [39]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=list('abcde'),
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-0.755512,0.184441,-0.150711,-0.668599,-0.177609
Steve,0.467969,0.160571,-2.285719,0.513607,0.326732
Wes,0.892516,NaN,NaN,0.403178,-1.147657
Jim,2.317518,-0.579703,-0.287207,0.194935,-2.038882
Travis,-0.021078,-0.147013,1.894022,-0.769496,0.888086


In [40]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f': 'orange'} # unused grouping keys are OK
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-0.819311,-0.748680
Steve,-1.772112,0.955271
Wes,0.403178,-0.255141
Jim,-0.092272,-0.301067
Travis,1.124525,0.719995


In [41]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [45]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions